# Introduction

1. I might be doing the scalar multiplies with the transpose -- if things don't work, check here


### See if it works on cartpole

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from agent import Agent
%matplotlib inline


#Environment
env = gym.make('CartPole-v0')
env.seed(0)
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n


#Agent
lr,gamma,tau, alpha = 0.0001, 0.99, 0.1, 0.001
clipnorm, verbose = False, False
agent = Agent(input_dim, output_dim, lr, gamma, tau, alpha, clipnorm, verbose)


#Train
EPISODES = 400
scores = []
for e in range(1,EPISODES+1):
    state = env.reset()
    state = agent.make_tensor(state)
    reward_sum = 0
    done = False
    while not done:
        
        #Do main step
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward_sum += reward
        next_state = agent.make_tensor(next_state)
        agent.remember(state[0],action,reward,next_state[0],done) #want to remember state as a vec
        
        #Learn
        if e > 100:
            agent.learn()
            
        state = next_state
        if done:
            break
    
    #Learn & print results
    scores.append(reward_sum)
    if e % 50 == 0:
        print '(episode, score) = ' + str((e,reward_sum))
plt.plot(scores)

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(episode, score) = (50, 32.0)
(episode, score) = (100, 28.0)


AttributeError: CriticV instance has no attribute 'soft_update_target_network'

### Debug the learning

In [50]:
#get batch
S2,A2,R2,S12,D2 = agent.get_batch()

#Get what we need for learning
Q2 = agent.criticQ.model.predict([S2,A2])
V2 = agent.criticV.target_model.predict(S12)  #value of NEXT state

Pi_vec = agent.actor.model.predict(S2)
Pi2 = np.sum(Pi_vec*A2,axis=1)          #now a batch of \pi(s_t,a_t)
Pi2 = np.array([[i] for i in Pi2])

#train actor
agent.actor.learn(S2,A2,Q2,V2)

#train critic
agent.criticQ.learn(S2,A2,R2,V2)

#train value funtion
agent.criticV.learn(S2,Q2,Pi2)

### CriticV 

In [46]:
#Define model
S = Input(shape=(input_dim,))
x = Dense(128, activation = 'relu')(S)
out = Dense(1, activation = 'linear')(x)
model = Model(inputs = S, outputs = out)

#Find terms in bracket
S_pl = model.input
Pi_pl = K.placeholder(shape=(None,1))
Q_pl = K.placeholder(shape=(None,1))
V_pl = model.output
#temp = V_pl - Q_pl + self.alpha*K.log(Pi_pl)

#Find gradient
pars = model.trainable_weights
grads = tf.gradients(V_pl, pars, V_pl - Q_pl + Pi_pl)  #scalar multiply by temp

#Do learning
#To get keras to apply updates given a custom gradients (i.e. run the above line) I had to alter the source
#Code. It was easy to do. See line X in the get_updates function.
#opt = Adam(self.lr)
#loss = grads  #placeholder, keras doesn't use it
#updates = opt.get_updates(loss = loss, params = pars, grads = grads)

#This function will apply updates when called
func = K.function(inputs = [S_pl, Q_pl, Pi_pl], outputs = [x for x in grads])
func([S2,Q2,Pi3])

[array([[ 1.51027110e-03, -1.45281553e-02, -1.27284112e-03,
         -1.99574931e-03,  0.00000000e+00,  2.11893637e-02,
          3.41017591e-03, -3.24681103e-02, -2.23310571e-02,
         -1.75426975e-02, -6.00003498e-03,  2.85510365e-02,
          2.99270195e-03,  1.86458963e-03, -1.27616641e-03,
         -1.37190530e-02,  1.12744514e-03, -1.00985505e-02,
          1.83073822e-02,  2.29555015e-02, -1.54627813e-03,
          3.32161374e-02,  1.62787270e-02, -2.91726906e-02,
          1.10129907e-03,  2.91941091e-02, -2.91204336e-03,
          6.46402710e-04,  3.11365281e-03, -1.24060910e-03,
          2.09052479e-04,  1.67605653e-02,  1.21758655e-02,
         -2.34417580e-02,  3.56843323e-03, -1.24385357e-02,
          2.86449143e-03, -2.12741550e-02, -2.21519498e-03,
          3.40363849e-03, -1.35363685e-03, -4.56820009e-03,
         -2.57358328e-03, -4.02094517e-03,  3.64861684e-03,
         -2.33322866e-02,  1.65733771e-04, -3.14151086e-02,
          2.39921384e-03, -1.95071660e-0

In [35]:
Pi3 = np.array([[i] for i in Pi2])
Pi3

array([[0.5052697 ],
       [0.4708679 ],
       [0.45295146]], dtype=float32)

In [40]:
V2 - Q2 + alpha*np.log(Pi3)

array([[-0.02717253],
       [ 0.17803136],
       [-0.01151956]], dtype=float32)

In [45]:
V2.shape, Q2.shape, Pi3.shape

((3, 1), (3, 1), (3, 1))

In [48]:
Pi2.shape == V2.shape

False

In [ ]:
as

### Critic network

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam 
from keras.optimizers import RMSprop
from keras import backend as K
from keras.utils import to_categorical
from keras.layers import Dense, Input, concatenate

input_dim, output_dim = 4,2
alpha = 0.1
lr = 0.1
gamma = 0.9

S = Input(shape=(input_dim,))
A = Input(shape=(output_dim,))
x = concatenate([S,A])
x = Dense(5, activation = 'relu')(x)
out = Dense(1, activation = 'linear')(x)
model = Model(inputs = [S,A], outputs = out)

#Input
S_pl, A_pl = model.input
Q_pl = model.output

#Find term in bracket
V_pl = K.placeholder(shape=(None,1))
R_pl = K.placeholder(shape=(None,1)) 
temp = Q_pl - R_pl + gamma*V_pl

#Find gradient
pars = model.trainable_weights
grads = tf.gradients(Q_pl,pars,temp)  #scalar multiply by temp

#Do update
opt = Adam(lr)
loss = grads
updates = opt.get_updates(loss = loss, params = pars, grads = grads)


func = K.function(inputs=[S_pl,A_pl,R_pl,V_pl],outputs=[x for x in grads], updates = updates)
func([S2,A2,R2,V2])

[array([[ 0.04806015,  0.15712519, -0.14912567,  0.05254675,  0.        ],
        [ 0.06267454,  0.9315323 , -1.1589466 ,  0.06852545,  0.        ],
        [-0.0440952 , -0.06870569,  0.03666694, -0.04821166,  0.        ],
        [-0.03239084, -1.1025333 ,  1.4233701 , -0.03541465,  0.        ],
        [ 1.2114152 , -1.7570772 ,  3.8302555 ,  1.3245054 ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]],
       dtype=float32),
 array([ 1.2114152, -1.7570772,  3.8302555,  1.3245054,  0.       ],
       dtype=float32),
 array([[-0.12519524],
        [-0.5022066 ],
        [-1.7734098 ],
        [-0.03578898],
        [ 0.        ]], dtype=float32),
 array([-4.0047693], dtype=float32)]

### Actor network

In [16]:
import tensorflow as tf
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


#Inputs
state_pl = model.input
action_pl = K.placeholder(shape=(None, output_dim))  #onehot
Q_pl = K.placeholder(shape=(None,1))
V_pl = K.placeholder(shape=(None,1))

#Find terms in bracket
pi_vec = model.output
pi = K.sum(pi_vec*action_pl,axis=1)    # get \pi(s_t, a_t) -- prob for specific action
entropy = alpha*K.log(pi)
temp = entropy - K.transpose(Q_pl) + K.transpose(V_pl)  #this is a row vec
temp = K.transpose(temp)      #turn it into col vec

#Find grad log(pi)
pi_pl = model.output
pars = model.trainable_weights
grads = tf.gradients( K.log(pi_pl), pars, temp)   #scalar multiply by temp

#Do learning
opt = Adam(lr)
loss = grads  #placeholder, I won't use it
updates = opt.get_updates(loss = grads, params = pars, grads = grads)

# To get keras to apply updates given a custom gradients (i.e. run the above line) I had to alter the source
# Code. It was easy to do. See line X in the get_updates function.

func =  K.function(inputs = [state_pl, action_pl, Q_pl, V_pl], outputs = grads + [temp])


#get batch
S2,A2,R2,S12,D = agent.get_batch()
Q2 = agent.criticQ.model.predict([S2,A2])
V2 = agent.criticV.target_model.predict(S12)
#Q2 = np.array([Q2.flatten()])
#V2 = np.array([V2.flatten()])

grads = func([S2,A2,Q2,V2])
grads,temp = grads[:-1], grads[-1] 